# Imports

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import datetime, time

# Exercise 1

#### explore data and find the max and min timestamps

In [3]:
#create 3 dataframe
a2q=pd.read_csv('data/sx-stackoverflow-a2q.txt',sep=' ', names=['u','v','t'])  #answers to questions
c2a=pd.read_csv('data/sx-stackoverflow-c2a.txt',sep=' ', names=['u','v','t'])  #comments to answers
c2q=pd.read_csv('data/sx-stackoverflow-c2q.txt',sep=' ', names=['u','v','t'])  #comments to questions

In [4]:
def findTimeMaxMin(data,max,min): #Retrieve maximum and minimum timestamps
    m = data['t'].min()
    M = data['t'].max()
    if m < min:
        min = m 
    if M > max:
        max = M
    return max, min

In [5]:
#Search maximum and minimum timestamps on top of 3 dataset files
max=0
min=1638921630 #2021/12/08
max,min=findTimeMaxMin(a2q,max,min)
max,min=findTimeMaxMin(c2a,max,min)
max,min=findTimeMaxMin(c2q,max,min)


print("\nMaximum timestamp is:",datetime.datetime.fromtimestamp(max).strftime("%Y-%m-%d"),sep=" ")
print("\nMinimum timestamp is:",datetime.datetime.fromtimestamp(min).strftime("%Y-%m-%d"),sep=" ")


Maximum timestamp is: 2011-06-09

Minimum timestamp is: 2008-08-01


##### I choose to *study* data from 2008/08/01 to 2009/08/01

In [6]:
t_start = min
t_end = datetime.datetime(2008, 12, 1, 0, 0)
t_end = time.mktime(t_end.timetuple())
t_end

1228089600.0

In [39]:
G = nx.DiGraph()

In [40]:
G.edges(data=True)

OutEdgeDataView([])

In [50]:
def createWeightedGraph(G, data, t_start, t_end):
    for index, line in data.iterrows():
        t = line['t']
        if t_start <= t < t_end :
            if G.has_edge(line['u'], line['v']) :
                G[line['u']][line['v']]['weight'] += 1
                G[line['u']][line['v']]['timestamp'].append(line['t'])
            else :
                G.add_edge(line['u'], line['v'], weight=1)
                G[line['u']][line['v']]['timestamp']=[line['t']]


In [ ]:
createWeightedGraph(G, a2q, t_start, t_end)
createWeightedGraph(G, c2a, t_start, t_end)
createWeightedGraph(G, c2q, t_start, t_end)

In [ ]:
#G.edges(data=True)

In [ ]:
#plt.clf()
#nx.draw(G)#, with_labels=True, with_weights = True)
#plt.show()

# Exercise 2

### Functionality 1 - Get the overall features of the graph


In [103]:
def F1_OverallFeatures(Graph):

    unique_nodes={}
    total_interactions=0
    average_interactions=0
    direct=True
    graph_density=0
    users = []
    
    for u,v,attr in (G.edges(data=True)):
        
        if (u in graphF1[v]) and (v in graphF1[u]):
            direct = False
        
        if u not in users: users.append(u)
        if v not in users: users.append(v)

        total_interactions+=1
    
    n_users = len(users)
    

    if n_users>0:
        average_interactions=total_interactions/n_users
        graph_density=total_interactions/(n_users*(n_users-1))
    
    print(n_users, total_interactions, direct, average_interactions, graph_density)
        

In [54]:
graphF1 = nx.DiGraph()
graphF1.edges(data=True)
createWeightedGraph(graphF1, a2q, t_start, t_end)

In [104]:
F1_OverallFeatures(graphF1)

268 874 False 3.2611940298507465 0.012214209849628264


{3, 4, 5}

True


True